<h2 >Sentiment Analysis</h2>

In [1]:
# pip install tweepy

     |████████████████████████████████| 94 kB 2.3 MB/s eta 0:00:011
     |████████████████████████████████| 62 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 151 kB 12.9 MB/s eta 0:00:01
ERROR: conda 4.14.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: oauthlib
    Found existing installation: oauthlib 3.1.0
    Uninstalling oauthlib-3.1.0:
      Successfully uninstalled oauthlib-3.1.0
Note: you may need to restart the kernel to use updated packages.


In [29]:
# pip install demoji

     |████████████████████████████████| 42 kB 1.8 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [30]:
import tweepy
import json
import time
import datetime
from datetime import date, timedelta

import pandas as pd
import re
import string

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import demoji
from emoji.unicode_codes import UNICODE_EMOJI

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sheenasalwan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
demoji.download_codes()

<ipython-input-31-b6fe2259445e>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


### Task 1: Data Extraction from Twitter API
---

In [15]:
api_key = '1D9l2cpV4EJaO1jBrdWU9zfoU' 
api_key_secret = 'qxyDTwZUABqd0sjaLe1Q3wQSxBhTjRexWbxDptu34fWt51ohpI'
access_token = '1572868505137188864-JSJF2tFizDiSvvMTUH5UhGjd1wOn5j'
access_token_secret = '6A562B7011LSkzmQs4TNZMLPO3USIw7mh5p89LlyKhOfD'

In [16]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [16]:
#api.search_tweets()

In [194]:
list_of_dicts = []
search_term = 'covid19 covid vaccine'
num_tweets = 11000

In [253]:
list_of_dates = []
today = date.today()
for i in range(-477,-470):
    target_date = (today + timedelta(days=i)).strftime("%Y-%m-%d")
    list_of_dates.append(target_date)

In [246]:
def get_tweets(search_term = search_term, num_tweets = num_tweets):
#     try:
        for end_date in list_of_dates:
            start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d") # Create 1-day windows for extraction
            tweet_count = len(list_of_dicts)

            for tweet in tweepy.Cursor(api.search_tweets,
                                       q=f'{search_term} since:{start_date} until:{end_date}',
                                      # q=search_term,
                                       lang = 'en',
                                       count = num_tweets,
                                       tweet_mode = 'extended').items(num_tweets):
                #if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                if 'RT @' not in tweet.full_text:
                    if tweet.lang == "en":
                        tweet_dict = {}
                        tweet_dict['username'] = tweet.user.name
                        tweet_dict['location'] = tweet.user.location
                        tweet_dict['text'] = tweet.full_text 
                        tweet_dict['hashtags'] = tweet.entities['hashtags']
                        dtx = tweet.created_at
                        tweet_dict['tweet_date'] = dtx.strftime("%m/%d/%Y, %H:%M:%S")
                        tweet_dict["retweeted"] = tweet.retweeted
                        tweet_dict["retweet_count"] =  tweet.retweet_count
                        tweet_dict["favorite_count"] = tweet.favorite_count
                        list_of_dicts.append(tweet_dict)
                        #json.dump(tweet_dict, outfile)
                        if tweet_count % 500 == 0:
                            print(f'Extracted tweet count = {tweet_count}')
                        tweet_count +=1
                        #print(tweet_count)
                        
            print(f'Completed extraction for {start_date} to {end_date}. Sleep for 15 mins')
            #time.sleep(900)
            print('Ready again')
#     except tweepy.TweepError:
#             print(tweepy.TweepError)
#             time.sleep(15 * 60)
#         

In [255]:
get_tweets()

Completed extraction for 2021-06-02 to 2021-06-03. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-03 to 2021-06-04. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-04 to 2021-06-05. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-05 to 2021-06-06. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-06 to 2021-06-07. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-07 to 2021-06-08. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-08 to 2021-06-09. Sleep for 15 mins
Ready again


In [2]:
# TO BE UNCOMMENTED FOR DATA EXTRACTION TESTING
# df = pd.DataFrame(list_of_dicts)
# df.head()
# ============
# s_set = {}

# for idx in range(0, len(df)):
#     s = df["tweet_date"][idx].split(",")[0]
#     #print(type(s))
#     if s in s_set.keys():
#         s_set[s] += 1
#     else:
#         s_set[s] = 1
    
# s_set
#=============
# df.to_csv("vaccination_all_tweets.csv")

### Task 2: Data Exploring
---

In [6]:
df_tweet = pd.read_csv("data/vaccination_all_tweets.csv")
df_tweet.tail(3)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
206964,1445954643419226114,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,26,0,0,False,2021-10-07 03:30:26,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False
206965,1445954599345475592,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,26,0,0,False,2021-10-07 03:30:15,18-44 #URBAN #Bengaluru #CovidVaccine Availabi...,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISH...",VaxBlr,0,0,False
206966,1445947047052333057,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,26,0,0,False,2021-10-07 03:00:15,45+ #URBAN #Bengaluru #CovidVaccine Availabili...,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISH...",VaxBlr,0,0,False


In [7]:
df_tweet.shape

(206967, 16)

In [8]:
df_tweet.describe()

,id,user_followers,user_friends,user_favourites,retweets,favorites
count,2.069670e+05,2.069670e+05,206967.000000,2.069670e+05,206967.000000,206967.000000
mean,1.402529e+18,9.356921e+04,983.137118,1.187433e+04,2.389091,10.334483
std,2.571008e+16,8.188852e+05,5390.710356,3.869035e+04,44.710131,160.316483
min,1.337728e+18,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
25%,1.380516e+18,5.900000e+01,25.000000,5.700000e+01,0.000000,0.000000
50%,1.405382e+18,3.390000e+02,227.000000,9.520000e+02,0.000000,0.000000
75%,1.422578e+18,1.521000e+03,786.000000,7.130000e+03,0.000000,2.000000
max,1.449092e+18,1.620117e+07,582461.000000,1.221784e+06,11288.000000,25724.000000


### Task 3: Text Cleaning and Processing
---

In [24]:
# lowercase and url removal

def process_tweet(text):
    text = text.lower()             # convert to lowercase
    text = re.sub(r"http\S+", "", text) # url removal
    return text

df_tweet["text"] = df_tweet["text"].apply(lambda text: process_tweet(text))
df_tweet["text"]

0         folks said daikon paste could treat cytokine s...
1         world wrong side history year hopefully bigges...
2         coronavirus sputnikv astrazeneca pfizerbiontec...
3         facts immutable senator even youre ethically s...
4         explain need vaccine borisjohnson matthancock ...
                                ...                        
206962    45 urban bengaluru covidvaccine availability 3...
206963    pincode 560011 sputnik v dose 1 100 slots age ...
206964    1844 bbmp bengaluru covidvaccine availability ...
206965    1844 urban bengaluru covidvaccine availability...
206966    45 urban bengaluru covidvaccine availability 0...
Name: text, Length: 206967, dtype: object

In [15]:
# Punctuation Removal

punctuation_removal = string.punctuation

def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', punctuation_removal))

df_tweet["text"] = df_tweet["text"].apply(lambda text: remove_punctuation(text))
df_tweet["text"]

0         same folks said daikon paste could treat a cyt...
1         while the world has been on the wrong side of ...
2         coronavirus sputnikv astrazeneca pfizerbiontec...
3         facts are immutable senator even when youre no...
4         explain to me again why we need a vaccine bori...
                                ...                        
206962    45 urban bengaluru covidvaccine availability f...
206963    pincode 560011\nsputnik v  dose 1 100 slots\n\...
206964    1844 bbmp bengaluru covidvaccine availability ...
206965    1844 urban bengaluru covidvaccine availability...
206966    45 urban bengaluru covidvaccine availability f...
Name: text, Length: 206967, dtype: object

In [ ]:
# lemmatization


In [22]:
# Stopword Removal

STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df_tweet["text"] = df_tweet["text"].apply(lambda text: remove_stopwords(text))
df_tweet["text"]

0         folks said daikon paste could treat cytokine s...
1         world wrong side history year hopefully bigges...
2         coronavirus sputnikv astrazeneca pfizerbiontec...
3         facts immutable senator even youre ethically s...
4         explain need vaccine borisjohnson matthancock ...
                                ...                        
206962    45 urban bengaluru covidvaccine availability 3...
206963    pincode 560011 sputnik v dose 1 100 slots age ...
206964    1844 bbmp bengaluru covidvaccine availability ...
206965    1844 urban bengaluru covidvaccine availability...
206966    45 urban bengaluru covidvaccine availability 0...
Name: text, Length: 206967, dtype: object

In [53]:
# emoji convert to text

def convert_emoji_to_text(tweet):
    tokens = tweet.split()
    for i, token in enumerate(tokens):
        if token in UNICODE_EMOJI["en"]: 
            emo_desc = demoji.findall(token)[token]
            new_rep = "_".join(emo_desc.split(":")[0].split())
            tokens[i] = new_rep
            
    return " ".join(tokens)

df_tweet["text"] = df_tweet["text"].apply(lambda text: convert_emoji_to_text(text))
df_tweet["text"]

0         folks said daikon paste could treat cytokine s...
1         world wrong side history year hopefully bigges...
2         coronavirus sputnikv astrazeneca pfizerbiontec...
3         facts immutable senator even youre ethically s...
4         explain need vaccine borisjohnson matthancock ...
                                ...                        
206962    45 urban bengaluru covidvaccine availability 3...
206963    pincode 560011 sputnik v dose 1 100 slots age ...
206964    1844 bbmp bengaluru covidvaccine availability ...
206965    1844 urban bengaluru covidvaccine availability...
206966    45 urban bengaluru covidvaccine availability 0...
Name: text, Length: 206967, dtype: object

### Task 4: Feature Engineering
---

In [57]:
# average length of sentence
def avg_len_sent(text):
    '''Calculates the average length of sentences, in tokens'''
    token_count = len(text.split())
    sent_count = text.count(". ") + 1
    if sent_count != 0:
        return token_count / sent_count
    else:
        return 0

In [58]:
df_tweet["avg_len_sent"] = df_tweet["text"].apply(lambda text:avg_len_sent(text))